# CMS 데이터 상,하한 이상감지(v 2.0)
## 1. 개요 
- CMS 데이터의 상,하한 값에 대한 이상을 감지하고 관련 담당자에게 메일을 송부함으로써 데이터 신뢰성 확보 및 회전설비 감시체계 강건화
- 기본컨셉은 CMS 데이터를 쿼리하고, 데이터를 정제한 후 정상기간(수리기간 제외) 데이터의 평균 및 표준편차(σ)를 구하여,  
통계학적으로 유의미한 이상한계값(3σ)을 초과 및 미만의 데이터는 이상값(Anomaly)로 판정하고, 이를 담당자에게 메일송부하는 시스템으로 구현 

## 2. Version History (현재 : 2.0ver 개발중)
###  [1.0 : 완료] 
 - [x] Data : 설비상태해석 시스템 시보데이터 추출(Excel Export) 
 - [x] 정제 : 누락되어있는 구간을 삭제하는 것이 아닌, 0으로 일괄처리
 - [x] 이상값 한계 설정 : 값이 0 이하(음수포함)인 경우를 제외하고, 나머지 데이터에서 평균, 표준편차산출
 - [x] 이상 판정 : Raw Data에서 상,하한을 넘어선 데이터를 이상값으로 판정
 - [x] 그래프 : Plotly를 이용하고, 이상값에 대한 그래프 별도표시
 - 관련기술 : Plotly Graph, 이상탐지기법(3α)

### [1.5 : 완료]
 - [x] Data : 설비상태해석 시스템 DB에서 직접 데이터 추출(Oracle Database 연결), 
 - [x] 5소결 201Belt Conveyor Data 쿼리(기간 : '20.1.1~12.31)
 - [x] 데이터 병렬화(2개이상 개소 데이터 추출)
 - 관련기술 : Oracle Database-Python 연결(pyodbc, oracle client 및 DNS 연결), 데이터 병렬처리

### [2.0 : 완료]
 - [x] Data : 설비상태해석 시스템 CMS 시보데이터 및 수리실적 데이터
 - [x] 그래프 : 수리,장애 실적구간은 별도색깔로 표시 및 labeling
 - 관련기술 : 구간별 그래프 별도표기 add_shape
 
### [2.5 : 완료]
 - [x] Data: 수리실적이 있는 구간은 이상판정 제외, 나머지 데이터로 이상기준 설정
 - [x] 그래프 : 수리실적이 있는 구간에서의 이상값은 별도 넘버링해서, 색상표시(회색에서 좀더 진하게)
 
### [3.0] 
 - Data : Local 데이터 추출(Excel Export)
 - 정제 : 기존 알고리즘에 적용할 수 있도록 데이터 정제

### [3.5]
 - Data 쿼리 : Local 데이터 DB에서 추출(Database 연결)

### [4.0]
 - Data 쿼리 : Local 데이터 1시간 단위 자동추출(Scheduling)

### [5.0] 
 - 이상치에 발생 시 일단위로 수합해서 담당자 메일송부

# 1. 라이브러리 Import

In [20]:
# Library import 
import pandas as pd
import numpy as np
import time
from datetime import datetime
import pyodbc
import os
import cufflinks as cf

from tqdm import tqdm 

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [3]:
# color pallette
cnf, dth, rec, act = '#393e46', '#ff2e63', '#21bf73', '#fe9801' 
DEFAULT_PLOTLY_COLORS=['rgb(31, 119, 180)', 'rgb(255, 127, 14)',
                       'rgb(44, 160, 44)', 'rgb(214, 39, 40)',
                       'rgb(148, 103, 189)', 'rgb(140, 86, 75)',
                       'rgb(227, 119, 194)', 'rgb(127, 127, 127)',
                       'rgb(188, 189, 34)', 'rgb(23, 190, 207)']

# 기본 폰트 설정
layout_font = {'font':dict(size=24,color='#60606e',family='Franklin Gothic' )}

# 2. ODBC로 Database 연결, Query문 정의

In [2]:
conn = pyodbc.connect('DSN=PKFAC_64;UID=PC629640P;PWD=welcom1!')
cursor = conn.cursor()
#CMS Data 추출 query문
query_values = '''select 
      c.LOCATION_DESCRIPTION as 공정명,
      a.EQP_DT_MSR_SNR_NM as 센서명,
      a.EQP_DT_MSR_SNR_ID as 센서ID,
      a.EQP_AST_CD as Asset,
      b.AC_DT as 발생일,
      b.EQP_MNTR_DT_A_V as 발생치,
      b.EQP_MNTR_DT_BAS_V1 as 주의치,
      b.EQP_MNTR_DT_BAS_V2 as 위험치


from POSFAC.TB_P20_MTFA120 a, -- CMS 센서현황
    POSFAC.TB_P20_MTFA140 b, -- CMS 시보실적
    POSFAC.TB_P20_EAMA120 c -- 공정(AREA)
       
  where 1=1
  
    AND A.EQP_RPR_OP_CD = B.EQP_RPR_OP_CD
    AND A.EQP_MNTR_DT_TP_TP = B.EQP_MNTR_DT_TP_TP
    AND A.EQP_DT_MSR_SNR_ID = B.EQP_DT_MSR_SNR_ID
    and a.EQP_AST_CD = '4K5311959'
    and a.EQP_RPR_OP_CD = c.LOCATION
    and b.AC_DT >= TO_DATE('2020-01-01', 'YYYY-MM-DD')
    and b.AC_DT < TO_DATE('2021-01-01', 'YYYY-MM-DD')
    and b.EQP_MNTR_DT_TP_TP = ?
      ORDER BY B.AC_DT ASC;
    '''
#휴지정보 추출 query문
query_huge = '''select 
    a.EQP_RPR_OP_CD as 공정코드,
    b.LOCATION_DESCRIPTION as 공정명,
    a.EQP_DORMANT_CD as 휴지코드,
    a.EQUIPMENT_DORMANT_START_DT as 시작일시,
    a.EQUIPMENT_DORMANT_END_DT as 종료일시,
    a.CREATION_TIMESTAMP as 생성일시

from
    POSFAC.TB_P20_PRCD420 a, --휴지정보
    POSFAC.TB_P20_EAMA120 b -- 공정정보(AREA)
    
where 1=1
    and a.EQP_RPR_OP_CD = b.LOCATION
    and a.EQUIPMENT_DORMANT_START_DT >= '2020-01-01'
    and a.EQUIPMENT_DORMANT_START_DT < '2021-01-01'
    --and b.LOCATION_DESCRIPTION LIKE
    and a.EQP_RPR_OP_CD = ?
    
order by a.EQP_RPR_OP_CD, a.EQUIPMENT_DORMANT_START_DT

'''

# 3. 쿼리문 Read
## 추후 조건 변경을 통해 여러 Data 추출
### 추출된 데이터들은 list형태로 저장

In [84]:
list = ['V1']
sqldata_list = []
sql_huge_list = []
for i in list:
    globals()['sqldata_{}'.format(i)] = pd.read_sql(query_values,conn, params=[i])
    sqldata_list.append('sqldata_{}'.format(i))
list_huge = ['KBF210']
for j in list_huge:
    globals()['sqldata_huge_{}'.format(j)] = pd.read_sql(query_huge, conn, params=[j])
    sql_huge_list.append('sqldata_huge_{}'.format(i))

# 4. 데이터 전처리
## 1) 결측치 : 0으로 대체
## 2) 휴지기간 데이터 제외(정상조업 Flag)

In [85]:
#결측치를 0으로 대체

def data_preprocess(cms_data,huge_data):
    cms_data.fillna(0) # 결측치는 0으로 대체
    N = cms_data['발생일'].count()   #CMS 데이터수량
    M = huge_data['시작일시'].count() #휴지정보 데이터 수량
    cms_data['휴지여부'] =0 
    # 휴지기간 내 발생된 CMS 데이터의 경우 '휴지여부' Column에 1로 표기
    for i in tqdm(range(N)):
        for j in range(M):
            if cms_data['발생일'][i] >= huge_data['시작일시'][j] and cms_data['발생일'][i] <= huge_data['종료일시'][j]:
                cms_data['휴지여부'][i] = 1
                break
            else:
                cms_data['휴지여부'][i] = 0 


In [86]:
data_preprocess(sqldata_V1, sqldata_huge_KBF210)

100%|█████████████████████████████████████████████████████████████████████████████| 6494/6494 [00:45<00:00, 142.72it/s]


# 5. 통계적 이상 기준설정
## 3σ 이상, 이하를 이상치로 판정
- 기준값 이상 - anomaly : 1    /     기준값 이하 - anomaly : 2

In [111]:
standard_index = sqldata_V1['휴지여부']==0
standard_data = sqldata_V1[standard_index]

def anomaly(sqldata, standard_data):
    sqldata['anomaly'] = 0
    sqldata['check'] = ""
    global upper_threshold, lower_threshold
    mean = standard_data['발생치'].mean()
    std = standard_data['발생치'].std()
    upper_threshold = mean + std*3
    lower_threshold = mean - std*3
    countN = sqldata['발생치'].count()
    for i in range(countN):
        if sqldata['휴지여부'][i] == 0:
            if sqldata['발생치'][i] >= upper_threshold:
                sqldata['anomaly'][i] = 1
                sqldata['check'][i] = "이상(기준값 이상)"
            elif sqldata['발생치'][i] <= lower_threshold:
                sqldata['anomaly'][i] = 2
                sqldata['check'][i] = "이상(기준값 이하)"
        else :
            if sqldata['발생치'][i] >= upper_threshold:
                sqldata['anomaly'][i] = 3
                sqldata['check'][i] = "이상(기준값 이상)"
            elif sqldata['발생치'][i] <= lower_threshold:
                sqldata['anomaly'][i] = 4
                sqldata['check'][i] = "이상(기준값 이하)"

In [112]:
anomaly(sqldata_V1,standard_data)

In [100]:
# 휴지기간 색깔표시
def huge_shade(sql_data):
    huge_cnt2 = sql_data['공정명'].count()
    
    for k in range(huge_cnt2):
        fig.add_shape(type='rect',
                     xref='x',
                     yref='y',
                     x0=sql_data['시작일시'][k],
                     y0=-0.5,
                     x1=sql_data['종료일시'][k],
                     y1=sqldata_V1['발생치'].max()+0.1,
                     line=dict(color="rgba(0,0,0,0)", width=8,),
              fillcolor='rgba(100,100,100,0.7)',
              layer='below')
    fig.show()
    

'''
fig.add_vrect(
    x0="2015-02-20", x1="2015-02-22",
    fillcolor="LightSalmon", opacity=0.5,
    layer="below", line_width=0,
)
'''

In [153]:
countN = sqldata_V1['발생치'].count()
fig = go.Figure()
marker1_color = [DEFAULT_PLOTLY_COLORS[3] if i == 1 or i==2 else DEFAULT_PLOTLY_COLORS[8] if i==3 or i==4 else DEFAULT_PLOTLY_COLORS[7] for i in sqldata_V1['anomaly']]
marker1_size = [8 if i == 1 or i == 2 else 5 for i in sqldata_V1['anomaly']]

fig.add_trace(go.Scatter(x= sqldata_V1['발생일'], y= sqldata_V1['발생치'],
                        mode='markers', marker=dict(color=marker1_color, size=marker1_size), name='5소결201BC'))
#상한값 경계
fig.add_shape(type="line", 
              x0= sqldata_V1['발생일'][0],
              x1= sqldata_V1['발생일'].iloc[-1],
              y0=upper_threshold, 
              y1=upper_threshold, 
              line=dict(
                  color="RoyalBlue",
                  width=2,
                  dash='dot',
                       )
             )
# 하한값 경계
fig.add_shape(type="line", 
              x0= sqldata_V1['발생일'][0],
              x1= sqldata_V1['발생일'].iloc[-1],
              y0=lower_threshold, 
              y1=lower_threshold, 
              line=dict(
                  color="RoyalBlue",
                  width=2,
                  dash='dot',
                       )
             )
fig.update_layout(title='<b> 진동값(속도) 추이 <b>')
fig.update_yaxes(range=[-0.5,sqldata_V1['발생치'].max()+0.1])
#fig.update_shapes(dict(xref='x',yref='y'))
huge_shade(sqldata_huge_KBF210)